# **CIS 5450 Final Project: Flight Cancellations**

Adam Gorka, Andrew Lukashchuk, Hassan Rizwan

*Due Date: **December 11th at 10:00PM EST**, 105 points total (graded out of 100)*

[Project Guidelines](https://docs.google.com/document/d/1o4XshJyIuNlm7_NeoE4RQmVfxWUcwA44bbMo9GbyhYs/edit)

[Project Demo](https://colab.research.google.com/drive/1u1O5IVfcgPxvPljJTIi2klpscE74Iubj?usp=sharing#scrollTo=toje9w6Eh3jI)


# **Part I:** Introduction and Setup

In our project, we intend to study data on flights in 2022. With this data, we want to find relationships on the cancellation of flights between various possible factors impacting the cancellation, including the flight date, airline, origin and destination city, distance, and year. It will be possible to see if there are any significant relationships between these factors and the flight being canceled, especially given the large number of entries in the dataset. Having a high number of data entries will prove useful in finding accurate relationships between the many different data points.

First, run the following cells to set up the notebook.

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from google.colab import drive
import os

In [ ]:
%%capture
!apt update
!pip install kaggle

# **Part II:** Preprocessing

We are using one Parquet for this project, `Combined_Flights_2022.parquet` from a Kaggle [dataset](https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022?select=Combined_Flights_2022.parquet). The dataset contains 61 columns and over 4 million flight entries.

In [ ]:
# Only run once per session

# Mount Google Drive
drive.mount('/content/drive')

# Create the kaggle directory
!mkdir ~/.kaggle

Mounted at /content/drive


In [ ]:
# Read the uploaded kaggle.json file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
# Download dataset
!!kaggle datasets download robikscube/flight-delay-dataset-20182022 -f Combined_Flights_2022.parquet

['Downloading Combined_Flights_2022.parquet.zip to /content',
 '',
 '  0% 0.00/102M [00:00<?, ?B/s]',
 '  6% 6.00M/102M [00:00<00:01, 60.5MB/s]',
 ' 13% 13.0M/102M [00:00<00:01, 63.9MB/s]',
 ' 25% 26.0M/102M [00:00<00:00, 91.2MB/s]',
 ' 36% 37.0M/102M [00:00<00:00, 98.6MB/s]',
 ' 46% 47.0M/102M [00:00<00:00, 78.8MB/s]',
 ' 56% 57.0M/102M [00:00<00:00, 66.5MB/s]',
 ' 63% 64.0M/102M [00:00<00:00, 65.9MB/s]',
 ' 74% 76.0M/102M [00:01<00:00, 80.2MB/s]',
 ' 86% 88.0M/102M [00:01<00:00, 90.9MB/s]',
 ' 96% 98.0M/102M [00:01<00:00, 83.4MB/s]',
 '',
 '100% 102M/102M [00:01<00:00, 80.5MB/s] ']

In [ ]:
# Unzip folder in Colab content folder
!unzip /content/Combined_Flights_2022.parquet.zip

Archive:  /content/Combined_Flights_2022.parquet.zip
  inflating: Combined_Flights_2022.parquet  


In [ ]:
# Load data into Pandas DataFrame
df = pd.read_parquet('Combined_Flights_2022.parquet')
df = df.dropna()
#df = pd.read_csv("Combined_Flights_2022.csv")

In [ ]:
# Print out the first five rows
df.head()

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0


In [ ]:
# See basic dataframe type info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3944916 entries, 0 to 590539
Data columns (total 61 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   FlightDate                               datetime64[ns]
 1   Airline                                  object        
 2   Origin                                   object        
 3   Dest                                     object        
 4   Cancelled                                bool          
 5   Diverted                                 bool          
 6   CRSDepTime                               int64         
 7   DepTime                                  float64       
 8   DepDelayMinutes                          float64       
 9   DepDelay                                 float64       
 10  ArrTime                                  float64       
 11  ArrDelayMinutes                          float64       
 12  AirTime                      

In [ ]:
# Basic Dataframe Summary
df.describe()

,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,...,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings
count,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,...,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06,3.944916e+06
mean,1.328088e+03,1.334120e+03,1.594013e+01,1.301277e+01,1.457636e+03,1.578307e+01,1.110075e+02,1.413464e+02,1.358624e+02,7.988929e+02,...,1.696645e+01,1.356426e+03,1.454810e+03,7.888369e+00,1.484449e+03,7.528486e+00,2.164715e-01,-6.256103e-02,3.667419e+00,4.562835e-06
std,4.899073e+02,5.057184e+02,5.216240e+01,5.316724e+01,5.429405e+02,5.198424e+01,6.996246e+01,7.199693e+01,7.185501e+01,5.931635e+02,...,9.483114e+00,5.076589e+02,5.376202e+02,6.647231e+00,5.178388e+02,5.524625e+01,4.118393e-01,2.487442e+00,2.326462e+00,6.408237e-03
min,1.000000e+00,1.000000e+00,0.000000e+00,-7.800000e+01,1.000000e+00,0.000000e+00,8.000000e+00,-4.800000e+01,1.400000e+01,3.100000e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.000000e+02,0.000000e+00,-2.000000e+00,1.000000e+00,0.000000e+00
25%,9.130000e+02,9.160000e+02,0.000000e+00,-5.000000e+00,1.046000e+03,0.000000e+00,6.000000e+01,8.900000e+01,8.300000e+01,3.680000e+02,...,1.100000e+01,9.320000e+02,1.043000e+03,4.000000e+00,1.102000e+03,-1.400000e+01,0.000000e+00,-1.000000e+00,2.000000e+00,0.000000e+00
50%,1.320000e+03,1.325000e+03,0.000000e+00,-2.000000e+00,1.459000e+03,0.000000e+00,9.400000e+01,1.240000e+02,1.190000e+02,6.430000e+02,...,1.500000e+01,1.338000e+03,1.456000e+03,6.000000e+00,1.510000e+03,-5.000000e+00,0.000000e+00,-1.000000e+00,3.000000e+00,0.000000e+00
75%,1.732000e+03,1.744000e+03,1.100000e+01,1.100000e+01,1.913000e+03,1.000000e+01,1.410000e+02,1.720000e+02,1.670000e+02,1.035000e+03,...,1.900000e+01,1.758000e+03,1.909000e+03,9.000000e+00,1.919000e+03,1.000000e+01,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00
max,2.359000e+03,2.400000e+03,7.223000e+03,7.223000e+03,2.400000e+03,7.232000e+03,7.270000e+02,6.900000e+02,7.640000e+02,5.095000e+03,...,2.210000e+02,2.400000e+03,2.400000e+03,2.900000e+02,2.359000e+03,7.232000e+03,1.000000e+00,1.200000e+01,1.100000e+01,9.000000e+00


In the dataframe info, we see that DayOfWeek is of type 'int64'. We'll start by mapping this to a string of type object for simplicity.

In [ ]:
# Check that the current DayOfWeek column includes all 7 days of the week and see if it uses 0-6 or 1-7
df['DayOfWeek'].unique()

array([1, 7, 6, 5, 2, 3, 4])

In [ ]:
# Given that it uses integers 1-7, map accordingly
day_mapping = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
df['DayOfWeekName'] = df['DayOfWeek'].map(day_mapping)

# **Part III:** Exploratory Analysis

Now that we a general idea of the structure of the dataframe, we want to begin by performing some Exploratory Analysis on the data to see what information the data can give us.

First we'll make a correlation matrix to see if any relationships or correlations between the various attributes stick out.

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# Display histograms of numerical columns
df.hist(figsize=(15, 12), bins=20)
plt.suptitle("Histogram of Numerical Columns", y=0.92)
plt.show()

In [ ]:
# Display count plots of categorical columns with only the top 50 values on the x-axis
categorical_columns = df.select_dtypes(include=["object"]).columns
for column in categorical_columns:
    # Get the top 50 values for the specified column
    top_50_values = df[column].value_counts().nlargest(50).index

    # Create a count plot with only the top 50 values on the x-axis
    plt.figure(figsize=(10, 6))
    sns.countplot(x=column, data=df[df[column].isin(top_50_values)], order=top_50_values)
    plt.title(f"Count Plot of {column} (Top 50)")

    # Improve x-axis label rotation for better readability
    plt.xticks(rotation=45, ha="right")

    plt.show()

In [ ]:
# Assuming you want to aggregate delays by date
df_aggregated = df.groupby('FlightDate').agg({'DepDelayMinutes': 'mean', 'ArrDelayMinutes': 'mean'}).reset_index()

# Visualize trends over time
plt.figure(figsize=(15, 6))
sns.lineplot(x='FlightDate', y='DepDelayMinutes', data=df_aggregated, label='Departure Delay')
sns.lineplot(x='FlightDate', y='ArrDelayMinutes', data=df_aggregated, label='Arrival Delay')
plt.title('Departure and Arrival Delays Over Time')
plt.xlabel('Flight Date')
plt.ylabel('Delay Minutes')
plt.legend()
plt.show()

In [ ]:
# Count plot of cancellation status
plt.figure(figsize=(8, 6))
sns.countplot(x='Cancelled', data=df)
plt.title('Count Plot of Cancellation Status')
plt.xlabel('Cancellation Status')
plt.ylabel('Count')
plt.show()

In [ ]:
# Bar plot of average departure delay by day of the week
plt.figure(figsize=(10, 6))
sns.barplot(x='DayOfWeekName', y='DepDelayMinutes', data=df, order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Average Departure Delay by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Departure Delay (Minutes)')
plt.xticks(rotation=45, ha="right")
plt.show()

In [ ]:
# Scatter plot of departure delay versus arrival delay
plt.figure(figsize=(10, 8))
sns.scatterplot(x='DepDelayMinutes', y='ArrDelayMinutes', data=df)
plt.title('Scatter Plot of Departure Delay versus Arrival Delay')
plt.xlabel('Departure Delay (Minutes)')
plt.ylabel('Arrival Delay (Minutes)')
plt.show()

In [ ]:
# Bar plot of average arrival delay by airline
plt.figure(figsize=(15, 8))
sns.barplot(x='Airline', y='ArrDelayMinutes', data=df)
plt.title('Average Arrival Delay by Airline')
plt.xlabel('Airline')
plt.ylabel('Average Arrival Delay (Minutes)')
plt.xticks(rotation=45, ha="right")
plt.show()

In [ ]:
# Drop unnecessary columns for modeling
df_model = df.drop(['FlightDate', 'Cancelled', 'Diverted', 'ArrDelayMinutes'], axis=1)

# Convert categorical variables to numerical using Label Encoding
label_encoder = LabelEncoder()
for column in df_model.select_dtypes(include='object').columns:
    df_model[column] = label_encoder.fit_transform(df_model[column])

# Create a binary target variable indicating whether the flight was delayed or not
df_model['Delayed'] = df_model['DepDelayMinutes'] > 0

# Separate features (X) and target variable (y)
X = df_model.drop(['Delayed', 'DepDelayMinutes'], axis=1)
y = df_model['Delayed']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model
rf_classifier.fit(X_train_imputed, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_imputed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print('\nConfusion Matrix:')
print(conf_matrix)
print('\nClassification Report:')
print(class_report)

In [ ]:
# Drop unnecessary columns for modeling
df_model_arr = df.drop(['FlightDate', 'Cancelled', 'Diverted', 'DepDelayMinutes'], axis=1)

# Convert categorical variables to numerical using Label Encoding
label_encoder = LabelEncoder()
for column in df_model_arr.select_dtypes(include='object').columns:
    df_model_arr[column] = label_encoder.fit_transform(df_model_arr[column])

# Create a binary target variable indicating whether the flight was delayed or not
df_model_arr['Delayed'] = df_model_arr['ArrDelayMinutes'] > 0

# Separate features (X) and target variable (y)
X = df_model_arr.drop(['Delayed', 'DepDelayMinutes'], axis=1)
y = df_model_arr['Delayed']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model
rf_classifier.fit(X_train_imputed, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_imputed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print('\nConfusion Matrix:')
print(conf_matrix)
print('\nClassification Report:')
print(class_report)

# **Part IV:** Data Wrangling and Staging

We are going to trim down unnessecary columns and data to make it cleaner and ready for modeling.

In [ ]:
# uh

# **Part V:** Modeling

Model

In [ ]:
# what

# **Part VI:** Feature Engineering

Test and training data, used to model to find difference

In [ ]:
Test =  2

# **Part VII:** More Modeling

Model test and training data, draw conclusions on predictability of flight cancellation

In [ ]:
#YEE

# **Part VIII:** Conclusion

Need to trim down dataset by dropping columns we won't be using and can then co further in depth with models on the relationships that appear to be there.